<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#MNIST-Classification-with-a-Long-Short-Term-Memory-(LSTM)-network" data-toc-modified-id="MNIST-Classification-with-a-Long-Short-Term-Memory-(LSTM)-network-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>MNIST Classification with a Long Short Term Memory (LSTM) network</a></span></li><li><span><a href="#Init-vars" data-toc-modified-id="Init-vars-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Init vars</a></span></li><li><span><a href="#Build-the-computational-graph" data-toc-modified-id="Build-the-computational-graph-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Build the computational graph</a></span><ul class="toc-item"><li><span><a href="#ELU-model" data-toc-modified-id="ELU-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>ELU model</a></span></li><li><span><a href="#Result" data-toc-modified-id="Result-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Result</a></span></li></ul></li><li><span><a href="#TensorBoard---Second-Run" data-toc-modified-id="TensorBoard---Second-Run-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TensorBoard - Second Run</a></span><ul class="toc-item"><li><span><a href="#Result" data-toc-modified-id="Result-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Result</a></span></li></ul></li><li><span><a href="#TensorBoard---Third-Run" data-toc-modified-id="TensorBoard---Third-Run-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>TensorBoard - Third Run</a></span><ul class="toc-item"><li><span><a href="#Result" data-toc-modified-id="Result-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Result</a></span></li></ul></li><li><span><a href="#TensorBoard---Fourth-Run" data-toc-modified-id="TensorBoard---Fourth-Run-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>TensorBoard - Fourth Run</a></span><ul class="toc-item"><li><span><a href="#Result" data-toc-modified-id="Result-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Result</a></span></li></ul></li><li><span><a href="#TensorBoard---Fifth-Run" data-toc-modified-id="TensorBoard---Fifth-Run-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>TensorBoard - Fifth Run</a></span><ul class="toc-item"><li><span><a href="#Result" data-toc-modified-id="Result-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Result</a></span></li></ul></li></ul></div>

<h1>MNIST Classification with a Long Short Term Memory (LSTM) network</h1>

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data

from functools import partial

Extracting ./datasets/mnist\train-images-idx3-ubyte.gz
Extracting ./datasets/mnist\train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist\t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist\t10k-labels-idx1-ubyte.gz


In [5]:
def resetGraph(seed= 10):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [6]:
def cleanLogs():
    os.system('rm -rf ./logs/dnnTensorBoard/')

# Init vars

The LSTM wants inputs of shape `[batchSize, timeSteps, numberOfInputs]`, and we have several thousand MNIST images of size 28 x 28 pixels.  

One way to think of this is a complete image is comprised of 28 rows of 28 pixels each.  If we were to step through the rows one by one and stack them up then the image would be more and more complete as time went by.  So our units of "time" will be the rows stacking together to create a complete image, and the number of inputs will be the number of pixels in the image row at that step in time (i.e. 28).  This gives us:

* batchSize       = number of observations (i.e. number of images in the mini batch)
* timeSteps       = number of rows we need to step through/stack up to make a complete image
* numberOfInputs  = the number of features in each row we are stepping through (i.e. also 28)

Additionally, we only care about the final output of the LSTM network which should give us the prediction of which numeral the image represents.  Other LSTM networks do care about the outputs of each LSTM cell (translating each word in a sentence for example), but that doesn't apply in our case.

Having said this we can continue with initializing the various variables we'll need:

In [8]:
# Setup vars for the MINST data set
timeSteps = 28
numOfInputs = 28

lstmUnits = 128
lr = 0.01
epochs = 20
batchSize = 50

numOfClasses = 10

mninst = input_data.read_data_sets("./datasets/mnist")

Extracting ./datasets/mnist\train-images-idx3-ubyte.gz
Extracting ./datasets/mnist\train-labels-idx1-ubyte.gz
Extracting ./datasets/mnist\t10k-images-idx3-ubyte.gz
Extracting ./datasets/mnist\t10k-labels-idx1-ubyte.gz


# Build the computational graph

In [9]:
# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir
logDir = './logs/mnistLSTM/runOne/train'

# Create place holders
x = tf.placeholder(tf.float32, shape = [None, timeSteps, numOfInputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None, numOfClasses], name = 'y')

# Create weights and bias tensors
with tf.name_scope("weightBias"):
    # Todo: Better init!!
    w = tf.Variable(tf.random_normal([lstmUnits, numOfClasses]))
    b = tf.Variable(tf.random_normal([numOfClasses]))


# Add the LSTM cells
with tf.name_scope("LSTM"):
    
    # Later in the code we'll make a call to tf.contrib.rnn.static_rnn
    # tf.contrib.rnn.static_rnn expects a length T list of inputs, each a Tensor of shape [batch_size, input_size]
    # So we need to convert our inputs of shape [batchSize, timeSteps, numberOfInputs] to [batch_size, input_size]
    #
    # https://www.tensorflow.org/versions/r1.1/api_docs/python/tf/contrib/rnn/static_rnn
    
    # https://www.tensorflow.org/api_docs/python/tf/unstack
    inputs = tf.unstack(x, num = timeSteps, axis = 1)
    
    # Create the basic LSTM cell
    # It does not allow cell clipping, a projection layer, and does not use peep-hole connections: it is the basic baseline.
    # https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell
    cell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
    
    # Add the cell to the RNN
    # https://www.tensorflow.org/versions/r1.1/api_docs/python/tf/contrib/rnn/static_rnn
    output, state = tf.contrib.rnn.static_rnn(cell, inputs, dtype = tf.float32)
    
    # We only care about the final output which should be the model's prediction
    yH = tf.matmul(ouput[-1], w) + b
    
# Add loss function
with tf.name_scope("loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits = yH, labels = y)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("optimizer"):
    opt = tf.train.AdamOptimizer(learning_rate = lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.equal(tf.argmax(yH, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

NameError: name 'ouput' is not defined

In [72]:
# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Create the TB writer and init
    trainWriter = tf.summary.FileWriter(logDir, sess.graph)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                merge = tf.summary.merge_all()
                trainWriter.add_summary(summary, counter)
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.9 Test Acc:  0.9039
10 Train Acc:  0.98 Test Acc:  0.9598
 
FINAL ::  Train Acc:  1.0 Test Acc:  0.9707


## ELU model

In [73]:
# Reset the TF CG
resetGraph()

# Set the TB logdir
logDir = './logs/dnnTensorBoard/elu/train'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # Use ELU
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.elu)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.elu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

In [74]:
# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Create the TB writer and init
    trainWriter = tf.summary.FileWriter(logDir, sess.graph)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                merge = tf.summary.merge_all()
                trainWriter.add_summary(summary, counter)
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.94 Test Acc:  0.9004
10 Train Acc:  1.0 Test Acc:  0.9471
 
FINAL ::  Train Acc:  1.0 Test Acc:  0.9612


## Result

![TB First Run](./images/tb-1st-run.png)

# TensorBoard - Second Run

Execute a single model, and capture loss and accuracy on the same plot.

In [11]:
# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/secondRun/train'
logDirTest = './logs/dnnTensorBoard/secondRun/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.96 Test Acc:  0.9021
10 Train Acc:  0.96 Test Acc:  0.9593
 
FINAL ::  Train Acc:  1.0 Test Acc:  0.97


## Result

![TB First Run](./images/tb-2nd-run.png)

# TensorBoard - Third Run

Execute a single model; capture loss and accuracy on the same plot; and capture metadata such as memory consumption.

In [11]:
# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/thirdRun/train'
logDirTest = './logs/dnnTensorBoard/thirdRun/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Generate and capture metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run(
                    [merge, opt], 
                    feed_dict = {x: xBatch, y: yBatch},
                    options=run_options,
                    run_metadata=run_metadata
                )
                
                # Write metadata
                trainWriter.add_run_metadata(run_metadata, 'step%03d' % counter)
                
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
            else:
                # Train as normal
                summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.9 Test Acc:  0.903
10 Train Acc:  1.0 Test Acc:  0.9594
 
FINAL ::  Train Acc:  0.98 Test Acc:  0.9701


## Result

![TB Third Run](./images/tb-3rd-run.png)

# TensorBoard - Fourth Run

Execute a single model multiple times with various hyperparameters; capture loss and accuracy on the same plot; and capture metadata such as memory consumption.

In [12]:
# Hyperparams
lr = 0.01
epochs = 20
batchSize = 50

# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/fourthRun/1/train'
logDirTest = './logs/dnnTensorBoard/fourthRun/1/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Generate and capture metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run(
                    [merge, opt], 
                    feed_dict = {x: xBatch, y: yBatch},
                    options=run_options,
                    run_metadata=run_metadata
                )
                
                # Write metadata
                trainWriter.add_run_metadata(run_metadata, 'step%03d' % counter)
                
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
            else:
                # Train as normal
                summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.88 Test Acc:  0.905
10 Train Acc:  0.96 Test Acc:  0.9598
 
FINAL ::  Train Acc:  0.94 Test Acc:  0.9709


In [13]:
# Alter hyperparams for second run
lr = 0.001
epochs = 30
batchSize = 100

# Reset the TF CG
resetGraph()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/fourthRun/2/train'
logDirTest = './logs/dnnTensorBoard/fourthRun/2/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Generate and capture metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run(
                    [merge, opt], 
                    feed_dict = {x: xBatch, y: yBatch},
                    options=run_options,
                    run_metadata=run_metadata
                )
                
                # Write metadata
                trainWriter.add_run_metadata(run_metadata, 'step%03d' % counter)
                
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
            else:
                # Train as normal
                summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.53 Test Acc:  0.552
10 Train Acc:  0.91 Test Acc:  0.8856
20 Train Acc:  0.94 Test Acc:  0.9062
 
FINAL ::  Train Acc:  0.91 Test Acc:  0.9156


## Result

![TB Fourth Run](./images/tb-4th-run.png)

# TensorBoard - Fifth Run

Execute a single model; capture loss and accuracy on the same plot; capture metadata such as memory consumption; and capture the each layer's bias, weights, and activations.

In [27]:
# Reset the TF CG
resetGraph()

# Clean away any old log files
cleanLogs()

# Set the TB logdir - We want two log dirs since we are going to be plotting two values on the same plot
logDirTrain = './logs/dnnTensorBoard/fifthRun/train'
logDirTest = './logs/dnnTensorBoard/fifthRun/test'

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("NN"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    # Create the first hidden layer
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    
    # Record the weights of bias of the layer and feed to the summary
    # This is the verbose method....
    with tf.variable_scope("hLayerOne", reuse=True):
        weights = tf.get_variable("kernel")
        bias = tf.get_variable("bias")
        tf.summary.histogram("layer1Weights", weights)
        tf.summary.histogram("layer1Bias", bias)
    
    
    # Create the second hidden layer
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    
    # Record the weights of bias of the layer and feed to the summary
    # And this time we'll do it the "easy" way
    tf.contrib.layers.summarize_activation(layer2)
    
    
    yH = tf.layers.dense(layer2, outputs, name = "yH")  
    # And the easy way again!  ;)
    tf.contrib.layers.summarize_activation(yH)
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    # Capture predictions and activation
    tf.summary.histogram("outputActivations", entropy)
    
    loss = tf.reduce_mean(entropy, name = "loss")
    # Capture loss
    tf.summary.scalar("loss", loss)
    
with tf.name_scope("train"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Capture accuracy
    tf.summary.scalar("accuracy", accuracy)

init = tf.global_variables_initializer()

# Execute the TF CG
counter = 0

for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)

with tf.Session() as sess:
    init.run()
    
    # Oddly enough if we want to write to values into the same plot we need TWO summary writers
    # One for training and one for test
    # Also notice we don't pass the 'sess.graph' arg to the 2nd summary writer instance
    # Also notice we don't call 'merge = tf.summary.merge_all()' other than this initial time
    # We add the summary values manually via calls to 'train_writer.add_summary(summaryObj, counterValue)'
    trainWriter = tf.summary.FileWriter(logDirTrain, sess.graph)
    testWriter = tf.summary.FileWriter(logDirTest)
    merge = tf.summary.merge_all()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):      
            
            counter += 1       
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            
            # Capture summary data every N steps
            if counter % 500 == 0:
                # Generate and capture metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run(
                    [merge, opt], 
                    feed_dict = {x: xBatch, y: yBatch},
                    options=run_options,
                    run_metadata=run_metadata
                )
                
                # Write metadata
                trainWriter.add_run_metadata(run_metadata, 'step%03d' % counter)
                
                # Manually add the test accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: mninst.test.images, y: mninst.test.labels})
                testWriter.add_summary(summary, counter)
                
                # Manually add the train accuracy summary value
                summary, acc = sess.run([merge, accuracy], feed_dict = {x: xBatch, y: yBatch})
                trainWriter.add_summary(summary, counter)
            else:
                # Train as normal
                summary, _ = sess.run([merge, opt], feed_dict = {x: xBatch, y: yBatch})
                
        
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

INFO:tensorflow:Summary name hLayerOne/kernel:0 is illegal; using hLayerOne/kernel_0 instead.
INFO:tensorflow:Summary name hLayerOne/bias:0 is illegal; using hLayerOne/bias_0 instead.
INFO:tensorflow:Summary name hLayerTwo/kernel:0 is illegal; using hLayerTwo/kernel_0 instead.
INFO:tensorflow:Summary name hLayerTwo/bias:0 is illegal; using hLayerTwo/bias_0 instead.
INFO:tensorflow:Summary name yH/kernel:0 is illegal; using yH/kernel_0 instead.
INFO:tensorflow:Summary name yH/bias:0 is illegal; using yH/bias_0 instead.
0 Train Acc:  0.96 Test Acc:  0.9056
10 Train Acc:  0.98 Test Acc:  0.9624
 
FINAL ::  Train Acc:  0.96 Test Acc:  0.9705


## Result

![TB First Run](./images/tb-5th-run.png)